In [40]:
import pandas as pd # Tabular data
from glob import glob # File pattern matching
import os # Operating System
import geopandas as gpd # Geospatial data
import re # Regular expressions
import math
from tqdm.auto import tqdm # Progress bars
from tqdm.contrib.concurrent import thread_map, process_map # Parallel operations
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 130)
df = pd.read_csv("meta.csv")
# Filter to just shapefiles that have the CRS column defined
df = df[df["CPS"]]
df["Pixel_ER"] = df.res.str.replace("(", "", regex=False).str.split(",").str[0].astype(float)
df = df[["filename", "matched_image", "Pixel_ER"]]
# Testing with BigBay
df = df[df.filename.str.contains("BigBay")]
df

,filename,matched_image,Pixel_ER
999,MaxarImagery/HighFreq/Southland/BigBay/Imagery/Shorelines/BigBay_07Mar2016.shp,MaxarImagery/HighFreq/Southland/BigBay/Imagery/Stack/BigBay_07MAR2016.tif,0.394305
1000,MaxarImagery/HighFreq/Southland/BigBay/Imagery/Shorelines/BigBay_04Sep2013.shp,MaxarImagery/HighFreq/Southland/BigBay/Imagery/Stack/BigBay_04SEP2013.tif,0.500000
1001,MaxarImagery/HighFreq/Southland/BigBay/Imagery/Shorelines/BigBay_31Jan2006.shp,MaxarImagery/HighFreq/Southland/BigBay/Imagery/Stack/BigBay_31JAN2006.tif,0.600000
1002,MaxarImagery/HighFreq/Southland/BigBay/Imagery/Shorelines/BigBay_27Mar2020.shp,MaxarImagery/HighFreq/Southland/BigBay/Imagery/Stack/BigBay_27MAR2020.tif,0.500000
1119,Retrolens/Southland/BigBay/Shorelines/BigBay_21FEB1967.shp,Retrolens/Southland/BigBay/Stack/BigBay_21FEB1967_mosaic.jp2,2.388996
1120,Retrolens/Southland/BigBay/Shorelines/BigBay_09MAR1985.shp,Retrolens/Southland/BigBay/Stack/BigBay_09MAR1985_mosaic.jp2,1.695647
1121,Retrolens/Southland/BigBay/Shorelines/BigBay_30DEC1984.shp,Retrolens/Southland/BigBay/Stack/BigBay_30DEC1984_mosaic.jp2,1.737656
1123,Retrolens/Southland/BigBay/Shorelines/BigBay_15FEB1952.shp,Retrolens/Southland/BigBay/Stack/BigBay_15FEB1952_mosaic.jp2,0.552018
1144,Retrolens/Southland/BigBay/Shorelines/BigBay_17MAR1979.shp,Retrolens/Southland/BigBay/Stack/BigBay_17MAR1979_mosaic.jp2,0.476924
1145,Retrolens/Southland/BigBay/Shorelines/BigBay_02APRL1953.shp,Retrolens/Southland/BigBay/Stack/BigBay_02APRL1953_mosaic.jp2,0.621867


Total UNCY:  
Total shoreline uncertainty (Et) is calculated as the root sum of squares of the pixel error (Ep), georeferencing error (Eg) and digitising error (Ed)  

$E_t = \sqrt{E_p^2 + E_g^2 + E_d^2}$

In [41]:
def get_scale(filename):
    # Find CSV for AOI
    bits = filename.split("/")
    terminator = min(bits.index("Stack") if "Stack" in bits else 1024, bits.index("Shorelines") if "Shorelines" in bits else 1024)
    bits = bits[:terminator]
    csv_path_pattern = f"ressci201900060-RNC2-Coastal/{'/'.join(bits)}/*.csv"
    csv_candidates = glob(csv_path_pattern)
    assert len(csv_candidates) == 1
    csv = pd.read_csv(csv_candidates[0], encoding='cp1252')
    year = re.search(r'(\d{4})', filename).group(1)
    scales = csv[csv.Date.str.contains(year)].Scale.unique()
    assert len(scales) == 1
    return scales[0]

def get_Georef_ER(scale):
  if scale < 20000:
    return 2.09
  elif scale < 30000:
    return 2.43
  else:
    return 2.9

CPS_error_lookup = {
  1: .43,
  2: .73,
  3: .97,
  4: 2.07,
  5: 8.59
}

for i, row in tqdm(df.iterrows(), total=len(df)):
    filename = row.filename
    shapefile = gpd.read_file(filename)
    source = shapefile.Source.unique()[0]
    if source == "MAX":
      photoscale = "VHR"
      Georef_ER = 1.17
    elif source == "LDS":
      photoscale = "VHR"
      Georef_ER = 0
    else:
      photoscale = get_scale(filename)
      Georef_ER = get_Georef_ER(photoscale)
    
    # Store inputs in shapefile
    shapefile["Photoscale"] = photoscale
    shapefile["Georef_ER"] = Georef_ER
    shapefile["Pixel_Er"] = row.Pixel_ER

    # Calculate Total_UNCY
    Ep = row.Pixel_ER
    Eg = Georef_ER
    for i, row in shapefile.iterrows():
        Ed = CPS_error_lookup[row.CPS]
        Et = math.sqrt(Ep**2 + Eg**2 + Ed**2)
        shapefile.loc[i, "Total_UNCY"] = Et
    display(shapefile)
    # To write results back to file:
    #shapefile.to_file(filename)

  0%|          | 0/12 [00:00<?, ?it/s]

,Id,Region,Site,Date,DSASDate,Digitiser,Scale,Notes,Source,CPS,Proxy,geometry,Photoscale,Georef_ER,Pixel_Er,Total_UNCY
0,0,Southland,BigBay,2016-03-07,07/03/2016,MT,1000,None,MAX,3,1,"LINESTRING (1208514.984 5079604.172, 1208518.688 5079604.701, 1208521.069 5079608.670, 1208523.715 5079612.639, 1208531.652 50...",VHR,1.17,0.394305,1.57012
1,0,Southland,BigBay,2016-03-07,07/03/2016,MT,1000,None,MAX,3,1,"LINESTRING (1208944.403 5079950.830, 1208947.049 5079957.444, 1208957.103 5079968.557, 1208967.951 5079982.051, 1208978.005 50...",VHR,1.17,0.394305,1.57012
2,0,Southland,BigBay,2016-03-07,07/03/2016,MT,1000,None,MAX,3,1,"LINESTRING (1209098.391 5080249.995, 1209096.909 5080258.885, 1209097.333 5080270.950, 1209099.873 5080276.453, 1209102.413 50...",VHR,1.17,0.394305,1.57012
3,0,Southland,BigBay,2016-03-07,07/03/2016,MT,1000,None,MAX,3,1,"LINESTRING (1209110.033 5080310.955, 1209109.398 5080320.691, 1209111.303 5080329.158, 1209113.843 5080337.836, 1209118.076 50...",VHR,1.17,0.394305,1.57012
4,0,Southland,BigBay,2016-03-07,07/03/2016,MT,1000,None,MAX,3,1,"LINESTRING (1209227.931 5080509.350, 1209231.953 5080518.240, 1209235.340 5080526.707, 1209235.340 5080535.364, 1209234.070 50...",VHR,1.17,0.394305,1.57012
5,0,Southland,BigBay,2016-03-07,07/03/2016,MT,1000,None,MAX,3,1,"LINESTRING (1209350.952 5080801.112, 1209355.397 5080807.886, 1209358.361 5080812.119, 1209360.054 5080818.046, 1209361.112 50...",VHR,1.17,0.394305,1.57012
6,0,Southland,BigBay,2016-03-07,07/03/2016,MT,1000,None,MAX,3,1,"LINESTRING (1209387.994 5080869.311, 1209391.592 5080875.238, 1209394.767 5080878.625, 1209397.307 5080884.340, 1209398.154 50...",VHR,1.17,0.394305,1.57012
7,0,Southland,BigBay,2016-03-07,07/03/2016,MT,1000,None,MAX,3,1,"LINESTRING (1209541.283 5081172.778, 1209545.728 5081179.340, 1209546.787 5081187.172, 1209549.750 5081195.003, 1209557.158 50...",VHR,1.17,0.394305,1.57012
8,0,Southland,BigBay,2016-03-07,07/03/2016,MT,1000,None,MAX,3,1,"LINESTRING (1209569.012 5081214.265, 1209565.413 5081218.710, 1209562.450 5081226.330, 1209559.487 5081229.294, 1209556.523 50...",VHR,1.17,0.394305,1.57012
9,0,Southland,BigBay,2016-03-07,07/03/2016,MT,1000,None,MAX,3,1,"LINESTRING (1210734.027 5084090.185, 1210719.274 5084107.098, 1210708.558 5084119.005, 1210698.239 5084133.689, 1210686.730 50...",VHR,1.17,0.394305,1.57012


,Id,Region,Site,Date,DSASDate,Digitiser,Scale,Notes,Source,CPS,Proxy,geometry,Photoscale,Georef_ER,Pixel_Er,Total_UNCY
0,0,Southland,BigBay,2013-09-04,04/09/2013,MT,1000,None,MAX,3,1,"LINESTRING (1208517.032 5079601.698, 1208523.316 5079612.943, 1208529.931 5079620.219, 1208537.538 5079624.188, 1208541.176 50...",VHR,1.17,0.5,1.599937
1,0,Southland,BigBay,2013-09-04,04/09/2013,MT,1000,None,MAX,3,1,"LINESTRING (1208946.783 5079951.048, 1208955.382 5079961.631, 1208961.996 5079970.561, 1208967.949 5079981.475, 1208974.895 50...",VHR,1.17,0.5,1.599937
2,0,Southland,BigBay,2013-09-04,04/09/2013,MT,1000,None,MAX,3,1,"LINESTRING (1209098.918 5080249.763, 1209098.257 5080262.331, 1209100.572 5080274.568, 1209103.879 5080286.474, 1209108.840 50...",VHR,1.17,0.5,1.599937
3,0,Southland,BigBay,2013-09-04,04/09/2013,MT,1000,None,MAX,4,1,"LINESTRING (1209111.817 5080310.948, 1209111.486 5080323.185, 1209117.439 5080340.714, 1209121.408 5080352.620, 1209128.684 50...",VHR,1.17,0.5,2.429774
4,0,Southland,BigBay,2013-09-04,04/09/2013,MT,1000,None,MAX,3,1,"LINESTRING (1209347.296 5080789.117, 1209353.250 5080802.016, 1209360.327 5080820.867, 1209368.595 5080836.412, 1209376.864 50...",VHR,1.17,0.5,1.599937
5,0,Southland,BigBay,2013-09-04,04/09/2013,MT,1000,None,MAX,4,1,"LINESTRING (1209561.411 5081232.732, 1209561.940 5081241.992, 1209566.703 5081247.813, 1209570.671 5081254.957, 1209574.905 50...",VHR,1.17,0.5,2.429774
6,0,Southland,BigBay,2013-09-04,04/09/2013,MT,1000,None,MAX,3,2,"LINESTRING (1209920.928 5082282.045, 1209917.488 5082287.601, 1209914.048 5082293.687, 1209913.519 5082301.360, 1209915.900 50...",VHR,1.17,0.5,1.599937
7,0,Southland,BigBay,2013-09-04,04/09/2013,MT,1000,None,MAX,3,1,"LINESTRING (1210737.702 5084087.389, 1210727.648 5084094.533, 1210720.240 5084106.174, 1210713.096 5084117.287, 1210703.836 50...",VHR,1.17,0.5,1.599937


,Id,Region,Site,Date,DSASDate,Digitiser,Scale,Notes,Source,CPS,Proxy,geometry,Photoscale,Georef_ER,Pixel_Er,Total_UNCY
0,0,SouthLand,BigBay,2006-01-31,31/01/2006,MT,1000,None,MAX,3,1,"LINESTRING (1208508.516 5079605.334, 1208516.123 5079608.311, 1208520.753 5079613.933, 1208524.722 5079618.894, 1208531.006 50...",VHR,1.17,0.6,1.633952
1,0,SouthLand,BigBay,2006-01-31,31/01/2006,MT,1000,None,MAX,3,1,"LINESTRING (1209137.234 5080237.326, 1209125.658 5080238.980, 1209108.129 5080244.602, 1209099.200 5080249.563, 1209095.892 50...",VHR,1.17,0.6,1.633952
2,0,SouthLand,BigBay,2006-01-31,31/01/2006,MT,1000,None,MAX,4,1,"LINESTRING (1209571.151 5081184.437, 1209571.482 5081194.690, 1209569.497 5081208.250, 1209566.190 5081214.864, 1209557.591 50...",VHR,1.17,0.6,2.452305
3,0,SouthLand,BigBay,2006-01-31,31/01/2006,MT,1000,None,MAX,4,1,"LINESTRING (1210314.158 5083587.560, 1210322.624 5083590.735, 1210326.858 5083596.556, 1210325.799 5083605.551, 1210317.862 50...",VHR,1.17,0.6,2.452305
4,0,SouthLand,BigBay,2006-01-31,31/01/2006,MT,1000,None,MAX,3,1,"LINESTRING (1210097.067 5084742.865, 1210122.202 5084730.297, 1210144.030 5084714.422, 1210165.197 5084688.625, 1210175.780 50...",VHR,1.17,0.6,1.633952


,Id,Region,Site,Date,DSASDate,Digitiser,Scale,Notes,Source,CPS,Proxy,geometry,Photoscale,Georef_ER,Pixel_Er,Total_UNCY
0,0,Southland,BigBay,2020-03-27,27/03/2020,MT,1000,None,MAX,3,1,"LINESTRING (1208513.399 5079602.927, 1208519.485 5079603.721, 1208522.395 5079606.896, 1208524.247 5079612.452, 1208529.539 50...",VHR,1.17,0.5,1.599937
1,0,Southland,BigBay,2020-03-27,27/03/2020,MT,1000,None,MAX,3,1,"LINESTRING (1208945.782 5079952.416, 1208950.809 5079960.618, 1208962.451 5079972.260, 1208969.859 5079982.049, 1208976.474 50...",VHR,1.17,0.5,1.599937
2,0,Southland,BigBay,2020-03-27,27/03/2020,MT,1000,None,MAX,2,1,"LINESTRING (1209100.617 5080250.470, 1209097.442 5080261.053, 1209098.764 5080271.901, 1209102.998 5080279.309, 1209103.792 50...",VHR,1.17,0.5,1.466901
3,0,Southland,BigBay,2020-03-27,27/03/2020,MT,1000,None,MAX,3,1,"LINESTRING (1209110.935 5080312.170, 1209111.729 5080323.018, 1209115.698 5080332.543, 1209124.429 5080346.566, 1209132.102 50...",VHR,1.17,0.5,1.599937
4,0,Southland,BigBay,2020-03-27,27/03/2020,MT,1000,None,MAX,3,1,"LINESTRING (1209243.492 5080506.640, 1209241.640 5080513.254, 1209240.052 5080519.075, 1209240.052 5080527.806, 1209243.227 50...",VHR,1.17,0.5,1.599937
5,0,Southland,BigBay,2020-03-27,27/03/2020,MT,1000,None,MAX,2,1,"LINESTRING (1209358.745 5080786.464, 1209356.416 5080794.719, 1209355.570 5080801.704, 1209358.745 5080812.075, 1209362.131 50...",VHR,1.17,0.5,1.466901
6,0,Southland,BigBay,2020-03-27,27/03/2020,MT,1000,None,MAX,4,1,"LINESTRING (1209389.436 5080867.151, 1209388.801 5080873.924, 1209393.035 5080883.449, 1209397.691 5080893.186, 1209402.136 50...",VHR,1.17,0.5,2.429774
7,0,Southland,BigBay,2020-03-27,27/03/2020,MT,1000,None,MAX,3,1,"LINESTRING (1209546.070 5081172.777, 1209547.763 5081177.434, 1209548.187 5081182.090, 1209548.822 5081186.535, 1209552.208 50...",VHR,1.17,0.5,1.599937
8,0,Southland,BigBay,2020-03-27,27/03/2020,MT,1000,None,MAX,3,1,"LINESTRING (1209610.290 5081376.253, 1209617.910 5081388.741, 1209624.683 5081401.018, 1209624.313 5081411.601, 1209626.853 50...",VHR,1.17,0.5,1.599937
9,0,Southland,BigBay,2020-03-27,27/03/2020,MT,1000,None,MAX,3,1,"LINESTRING (1209913.404 5082293.725, 1209909.277 5082296.107, 1209906.260 5082304.044, 1209906.419 5082313.252, 1209910.547 50...",VHR,1.17,0.5,1.599937


...

,Id,Region,Site,Date,DSASDate,Digitiser,Scale,Notes,Source,CPS,Proxy,geometry,Photoscale,Georef_ER,Pixel_Er,Total_UNCY
0,0,Southland,BigBay,1952-02-15,15/02/1952,MT,2000,None,RL,3,1.0,"LINESTRING (1208520.913 5079608.313, 1208545.254 5079625.511, 1208551.604 5079631.861, 1208557.193 5079630.640, 1208570.412 50...",15900,2.09,0.552018,2.369330
1,0,Southland,BigBay,1952-02-15,15/02/1952,MT,2000,None,RL,4,1.0,"LINESTRING (1209198.545 5080179.917, 1209187.961 5080188.515, 1209176.055 5080196.453, 1209164.810 5080205.713, 1209150.258 50...",15900,2.09,0.552018,2.992946
2,0,Southland,BigBay,1952-02-15,15/02/1952,MT,2000,None,RL,3,NaN,"LINESTRING (1210420.022 5084522.928, 1210422.668 5084509.699, 1210427.960 5084499.116, 1210435.368 5084486.945, 1210445.423 50...",15900,2.09,0.552018,2.369330


,Id,Region,Site,Date,DSASDate,Digitiser,Scale,Notes,Source,CPS,Proxy,geometry,Photoscale,Georef_ER,Pixel_Er,Total_UNCY
0,0,Southland,BigBay,1079-03-17,17/03/1079,MT,1250,None,RL,3,1,"LINESTRING (1208513.561 5079609.039, 1208528.907 5079620.681, 1208548.486 5079636.556, 1208571.770 5079625.973, 1208600.345 50...",15000,2.09,0.476924,2.352968
1,0,Southland,BigBay,1079-03-17,17/03/1079,MT,1250,None,RL,4,1,"LINESTRING (1209219.153 5080209.645, 1209182.641 5080218.111, 1209149.436 5080216.392, 1209117.686 5080202.104, 1209104.192 50...",15000,2.09,0.476924,2.980009
2,0,Southland,BigBay,1079-03-17,17/03/1079,MT,1250,None,RL,2,1,"LINESTRING (1210100.132 5084743.706, 1210126.722 5084727.434, 1210139.026 5084713.940, 1210157.679 5084696.875, 1210168.394 50...",15000,2.09,0.476924,2.264610


,Id,Region,Site,Date,DSASDate,Digitiser,Scale,Notes,Source,CPS,Photoscale,Georef_ER,Pixel_Er,Total_UNCY,Proxy,geometry
0,0,Southland,BigBay,1953-04-02,2/04/1953,MT,2000,None,RL,3,18200,2.09,0.621867,2.386571,1,"LINESTRING (1208542.227 5079629.664, 1208548.577 5079633.632, 1208566.039 5079620.932, 1208586.677 5079614.582, 1208613.664 50..."
1,0,Southland,BigBay,1953-04-02,2/04/1953,MT,2000,None,RL,5,18200,2.09,0.621867,8.862444,1,"LINESTRING (1209199.797 5080178.490, 1209187.891 5080188.412, 1209175.588 5080195.556, 1209153.363 5080213.018, 1209146.616 50..."
2,0,Southland,BigBay,1953-04-02,2/04/1953,MT,2000,None,RL,4,18200,2.09,0.621867,3.006613,1,"LINESTRING (1210202.063 5083043.106, 1210201.071 5083051.044, 1210204.047 5083061.958, 1210203.386 5083071.549, 1210205.039 50..."
3,0,Southland,BigBay,1953-04-02,2/04/1953,MT,2000,None,RL,3,18200,2.09,0.621867,2.386571,1,"LINESTRING (1210157.150 5084697.510, 1210170.379 5084683.222, 1210187.841 5084680.047, 1210226.471 5084667.347, 1210256.633 50..."


,Id,Region,Site,Date,DSASDate,Digitiser,Scale,Notes,Source,CPS,Proxy,geometry,Photoscale,Georef_ER,Pixel_Er,Total_UNCY
0,0,Southland,BigBay,1988-02-18,18/02/1988,MT,1000,None,RL,3,1,"LINESTRING (1208513.359 5079606.099, 1208520.238 5079614.566, 1208529.763 5079623.032, 1208537.172 5079625.678, 1208541.934 50...",15000,2.09,0.530837,2.364485
1,0,Southland,BigBay,1988-02-18,18/02/1988,MT,1000,None,RL,4,1,"LINESTRING (1209073.487 5080244.915, 1209068.526 5080247.891, 1209063.234 5080257.482, 1209066.211 5080273.688, 1209082.086 50...",15000,2.09,0.530837,2.989112
2,0,Southland,BigBay,1988-02-18,18/02/1988,MT,1000,None,RL,3,1,"LINESTRING (1210102.236 5084740.800, 1210139.278 5084713.681, 1210159.122 5084696.483, 1210180.288 5084681.269, 1210210.054 50...",15000,2.09,0.530837,2.364485


,Id,Region,Site,Date,DSASDate,Digitiser,Scale,Notes,Source,CPS,Proxy,geometry,Photoscale,Georef_ER,Pixel_Er,Total_UNCY
0,0,Southland,BigBay,1975-02-17,17/02/1975,MT,1100,None,RL,3,1,"LINESTRING (1208518.397 5079609.368, 1208545.516 5079636.488, 1208562.053 5079643.102, 1208573.298 5079631.858, 1208593.803 50...",25000,2.43,0.771031,2.727689
1,0,Southland,BigBay,1975-02-17,17/02/1975,MT,1100,None,RL,4,1,"LINESTRING (1209219.941 5080207.327, 1209207.373 5080211.958, 1209193.482 5080217.249, 1209182.237 5080217.911, 1209162.394 50...",25000,2.43,0.771031,3.283944
2,0,Southland,BigBay,1975-02-17,17/02/1975,MT,1100,None,RL,2,1,"LINESTRING (1210101.763 5084739.539, 1210116.580 5084730.807, 1210135.101 5084716.520, 1210149.918 5084704.878, 1210166.322 50...",25000,2.43,0.771031,2.651846
